In [ ]:
# ===============================================================
# This model intends to estimate the human finger forces using CNN
# based on the coloration features. The FingerNet is based on 
# MobileNet V2 by google.
# ===============================

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import os

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential

import matplotlib.pyplot as plt
import numpy as np
import time

In [ ]:
# ===============================
# Building the prediction model 
# on top of the base model 
# ===============================
def buil_model(width, height, depth, classes):
    # initialize the input shape and channels dimension to be
    # "channels last" ordering
    IMG_SHAPE = (height, width, depth)
    chanDim = -1
    base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                                include_top=False,
                                                weights='imagenet')
    # Show a summary of the model. Check the number of trainable parameters
    base_model.trainable = False
    # global average layer (also faltten the output of MobileNet)
    global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
    # first FC layer with droupout
    linear_layer = tf.keras.layers.Dense(1024, activation='relu')
    dropout_layer = tf.keras.layers.Dropout(0.5)
    # prediction layer with 3 outputs
    prediction_layer = tf.keras.layers.Dense(classes, activation= 'linear')
    # build the model using Keras' Sequential API
    model = Sequential([
        # MobileNet => avgPOOL => FC => RELU => DROPOUT => FC => LINEAR
        base_model,
        global_average_layer,
        linear_layer,
        dropout_layer,
        prediction_layer
    ])
    # return the constructed network architecture
    return model

In [ ]:
# ===============================
# Solver Parameters 
# ===============================
# initialize the number of epochs to train for, batch size, and initial learning rate
EPOCHS = 5
BS = 32
INIT_LR = 1e-3

loss_object = tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.NONE)
opt = tf.keras.optimizers.RMSprop(lr=INIT_LR, momentum=0.9)

train_loss = tf.keras.metrics.MeanSquaredError(name='train_loss')
train_accuracy = tf.keras.metrics.RootMeanSquaredError(name='train_accuracy')
# test_loss = tf.keras.metrics.Mean(name='test_loss')
# test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [ ]:
# ===============================
# Train function
# ===============================
@tf.function
def train_step(image, force):
    # keep track of our gradients
    with tf.GradientTape() as tape:
        # make a prediction using the model and then calculate the loss
        force_pred = model(image)
        loss = loss_object(force, force_pred)
    # calculate the gradients using our tape and then update the model weights
    grads = tape.gradient(loss, model.trainable_variables)
    opt.apply_gradients(zip(grads, model.trainable_variables))
    train_loss(loss)
    train_accuracy(force, force_pred)

In [ ]:
# ===============================
# Train the model
# ===============================
# build our model and initialize our optimizer
print("[INFO] creating model...")
model = buil_model(224, 224, 3, 3)
# model.summary()

# compute the number of batch updates per epoch
numUpdates = int(train_image.shape[0] / BS)

# loop over the number of epochs
for epoch in range(0, EPOCHS):
	# show the current epoch number
	print("[INFO] starting epoch {}/{}...".format(epoch + 1, EPOCHS), end="")
	sys.stdout.flush()
	epochStart = time.time()
    # loop over the data in batch size increments
	for i in range(0, numUpdates):
		# determine starting and ending slice indexes for the current batch
		start = i * BS
		end = start + BS
        # take a step
		train_step(train_image[start:end], train_image[start:end])
        
	# show timing information for the epoch
	epochEnd = time.time()
	elapsed = (epochEnd - epochStart) / 60.0
    template = 'Epoch {}, Loss: {}, Accuracy: {}'
	print("took {:.4} minutes".format(elapsed))
    print (template.format(epoch+1, train_loss.result(), train_accuracy.result()*100,)
    

In [ ]:
# img_path = 'dog.jpg'
# img = image.load_img(img_path, target_size=(224, 224))
# x = image.img_to_array(img)
# x = np.expand_dims(x, axis=0)
# # x = preprocess_input(x)
